# Lab 6 - Multiple Linear Regression

In [ ]:
import pandas as pd
import numpy as np 
from plotnine import ggplot, aes, geom_bar, stat_qq, stat_qq_line, position_dodge, geom_col, theme_minimal
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

## Question Set 1

Use the same cars2010 dataset you used in lab 4. This dataset has variables pertaining to fuel economy of various cars. Do not create a training and test dataset; just use the whole cars2010 dataset for analysis. The cars2011 and cars2012 datasets will be used at later time periods. 

In [ ]:
cars = pd.read_csv("https://raw.githubusercontent.com/IAA-Faculty/statistical_foundations/refs/heads/master/cars2010.csv")

### Question 1a

Run a regression predicting the FE variable using all the remaining variable. Some of these predictor variables are coded as numeric, but should be treated as categorical. The only numeric predictor should be EngDispl. All remaining predictors are categorical. What is the F-statistic from the global F-test? Round to 2 decimals.

In [ ]:
cars.columns

In [ ]:
fe_mlr = smf.ols("FE ~ EngDispl + C(NumCyl) + C(Transmission) + C(AirAspirationMethod) + C(NumGears) + C(TransLockup) + C(TransCreeperGear) + C(DriveDesc) + C(IntakeValvePerCyl) + C(ExhaustValvesPerCyl) + C(CarlineClassDesc) + C(VarValveTiming) + C(VarValveLift)", data = cars).fit()
fe_mlr.summary()

Answer: 95.55

### Question 1b

What percentage of variability can be explained by this model? Round to 1 decimal (remember, the question asks for what percentage)

Answer: 83.3

### Question 1c

Is the average effect that EngDispl has on FE significant?

Answer: at p < 0.001, yes

### Question 1d

What is the correct interpretation of the EngDispl coefficient?

Answer: When holding all other predictors fixed, FE decreases, on average, by 2.25 for each 1 unit increase of EngDispl

### Question 1e

Trying to evaluate categorical variables in traditional linear regression output can be difficult because the p-values are for each categorical dummy variable as a whole; you need a global p-value for each categorical variable. Use the anova_lm() function on your linear regression object to get the p-values for each global categorical variable. Which of the variables has the highest p-value?

In [ ]:
sms.anova_lm(fe_mlr, typ=2)

Answer: VarValveTiming

### Question 1f

Is the answer in part e) significant?

Answer: at p = 0.58, no

### Question 1g

Rerun the preceding model, but remove the variable from part e) (regardless of significance). Compare with the preceding model. Based on the global F-test, r-squared value, and adjusted r-squared value, is there a large difference between this model and the preceeding model?

In [ ]:
fe_mlr_reduced = smf.ols("FE ~ EngDispl + C(NumCyl) + C(Transmission) + C(AirAspirationMethod) + C(NumGears) + C(TransLockup) + C(TransCreeperGear) + C(DriveDesc) + C(IntakeValvePerCyl) + C(ExhaustValvesPerCyl) + C(CarlineClassDesc) + C(VarValveLift)", data = cars).fit()
fe_mlr_reduced.summary()

Answer: no

### Question 1h

Now, rerun the preceding model (from part g), but eliminate the variables with the highest p-value (determined using the same method from part e) and rerunning the regression until you only have variables significant at the 0.008 level. Remember to run the model after EACH variable you remove as the p-value might change by removing a variable.

Did the global F-statistic, r-squared, and adjusted r-squared values changed largely (hint for decision on F-stat: did it change the p-value much)?

In [ ]:
sms.anova_lm(fe_mlr_reduced, typ=2)

In [ ]:
fe_mlr_reduced_2 = smf.ols("FE ~ EngDispl + C(NumCyl) + C(Transmission) + C(AirAspirationMethod) + C(NumGears) + C(TransLockup) + C(DriveDesc) + C(IntakeValvePerCyl) + C(ExhaustValvesPerCyl) + C(CarlineClassDesc) + C(VarValveLift)", data = cars).fit()
sms.anova_lm(fe_mlr_reduced_2, typ=2)

In [ ]:
fe_mlr_reduced_3 = smf.ols("FE ~ EngDispl + C(NumCyl) + C(Transmission) + C(AirAspirationMethod) + C(NumGears) + C(DriveDesc) + C(IntakeValvePerCyl) + C(ExhaustValvesPerCyl) + C(CarlineClassDesc) + C(VarValveLift)", data = cars).fit()
sms.anova_lm(fe_mlr_reduced_3, typ=2)

In [ ]:
fe_mlr_reduced_4 = smf.ols("FE ~ EngDispl + C(NumCyl) + C(Transmission) + C(NumGears) + C(DriveDesc) + C(IntakeValvePerCyl) + C(ExhaustValvesPerCyl) + C(CarlineClassDesc) + C(VarValveLift)", data = cars).fit()
sms.anova_lm(fe_mlr_reduced_4, typ=2)

In [ ]:
fe_mlr_reduced_5 = smf.ols("FE ~ EngDispl + C(NumCyl) + C(Transmission) + C(NumGears) + C(DriveDesc) + C(IntakeValvePerCyl) + C(CarlineClassDesc) + C(VarValveLift)", data = cars).fit()
sms.anova_lm(fe_mlr_reduced_5, typ=2)

In [ ]:
fe_mlr_reduced_5.summary()

Answer: No

## Question Set 2

Using the bike data from previous labs, you want to build a couple of different models and see which one is better. We will learn in later lectures how to do this with test datasets, but for now, we will only do this with trianing data. First, we need to split the data into training and test. Run the following code to get the training and test split (since the data is randomly divided, use the same seed or we will not end up with the same data, thus we will not necessarily get the same results).

seed = 123
np.random.seed(seed)

bike['id'] = bike.index + 1
train = bike.sample(frac=0.7, random_state=seed)
test = bike[~bike['id'].isin(train['id'])]

In [ ]:

bike = pd.read_csv("https://raw.githubusercontent.com/IAA-Faculty/statistical_foundations/refs/heads/master/bike.csv")

seed = 123
np.random.seed(seed)

bike['id'] = bike.index + 1
train = bike.sample(frac=0.7, random_state=seed)
test = bike[~bike['id'].isin(train['id'])]

### Question 2a

How many observations are in the train dataset?

In [ ]:
train.shape

### Question 2b

How many observations are in the test dataset?

In [ ]:
test.shape

### Question 2c

Since you know that temp and atemp are highly correlated, you can't decide which variable is better to predict number of users (cnt). You probably shouldn't use both in your model since you will have multicollinearity issues, so one strategy is to build two models and compare them to see which predicts better. 

Using the training data, build one model with the following variables:
actual temperature (temp), humidity (hum), and wind speed (windspeed).

What is the adjusted r-squared values for this model? (Round to 3 decimals)

In [25]:
bike_model_1 = smf.ols("cnt ~ temp + hum + windspeed", data = bike).fit()
bike_model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cnt   R-squared:                       0.251
Model:                            OLS   Adj. R-squared:                  0.251
Method:                 Least Squares   F-statistic:                     1945.
Date:                Wed, 09 Jul 2025   Prob (F-statistic):               0.00
Time:                        12:32:44   Log-Likelihood:            -1.1253e+05
No. Observations:               17379   AIC:                         2.251e+05
Df Residuals:                   17375   BIC:                         2.251e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    175.8100      6.187     28.416      0.000     163.683     187.937
temp         362.5344      6.205     58.427      0.000     350.372     374.697
hum         -273.4651      6.469    -42.270      0.000    -286.146    -260.784
windspeed     26.3198     10.180      2.585      0.010       6.366      46.274
==============================================================================
Omnibus:                     3606.701   Durbin-Watson:                   0.417
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7255.109
Skew:                           1.244   Prob(JB):                         0.00
Kurtosis:                       4.957   Cond. No.                         12.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Question 2d

Is the global F-test significant?

Answer: At p < 0.01, yes

### Question 2e

Which variables are significant at the .01 significance level?

Answer: At p < 0.001, both temp and humidity are. At p = 0.01, windspeed is arguably significant

### Question 2f

Build a second model wih the following variables: feeling temperature (atemp), humidity (hum), and wind speed (windspeed). What is the adjusted r-squared value? (round to 3 decimals)

In [ ]:
bike_model_2 = smf.ols("cnt ~ atemp + hum + windspeed", data = bike).fit()
bike_model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cnt   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.253
Method:                 Least Squares   F-statistic:                     1964.
Date:                Wed, 09 Jul 2025   Prob (F-statistic):               0.00
Time:                        12:35:55   Log-Likelihood:            -1.1250e+05
No. Observations:               17379   AIC:                         2.250e+05
Df Residuals:                   17375   BIC:                         2.250e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    158.6952      6.335     25.049      0.000     146.277     171.113
atemp        409.2251      6.952     58.864      0.000     395.598     422.852
hum         -275.8632      6.458    -42.716      0.000    -288.522    -263.205
windspeed     47.8602     10.191      4.697      0.000      27.886      67.835
==============================================================================
Omnibus:                     3588.025   Durbin-Watson:                   0.420
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7231.830
Skew:                           1.236   Prob(JB):                         0.00
Kurtosis:                       4.968   Cond. No.                         12.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

: 

### Question 2g

Is the global F-test significant?

Answer: At p < 0.01, yes

### Question 2h

Which variables are significant at the .01 significance level?

Answer: At p < 0.001, temp, humidity, and windspeed are all significant